In [1]:
import numpy as np
import tensorflow as tf
import gym
import mujoco_py

np.random.seed(2)
tf.set_random_seed(2)  # reproducible

# Superparameters
OUTPUT_GRAPH = False
MAX_EPISODE = 50000
DISPLAY_REWARD_THRESHOLD = -10  # renders environment if total episode reward is greater then this threshold
MAX_EP_STEPS = 200 # maximum time step in one episode
RENDER = False # rendering wastes time
GAMMA = 0.9     # reward discount in TD error
ALPHA = 2
LR_A = 0.0005    # learning rate for actor
LR_C = 0.0005    # learning rate for critic
# env = gym.make('MountainCarContinuous-v0')
env = gym.make('Pendulum-v0')
env.seed(1)  # reproducible

N_F = env.observation_space.shape[0]
action_res = [41]
N_A = action_res[0]
u = np.linspace(env.action_space.low[0], env.action_space.high[0], num=action_res[0])
action_map = u

[2018-06-06 15:21:21,520] Making new env: Pendulum-v0


In [2]:
class ExpReplay(object):
    def __init__(self, memory_size, state_dim, act_dim, batch_size):
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0
    def fifo(self, state, action, reward, next_state):
        self.states[self.current] = state
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.next_states[self.current] = next_state
        self.current = (self.current + 1) % self.memory_size
        self.count = self.count + 1
    def add_trajectory(self, states, actions, rewards, next_states):
        num = len(observes)
        for i in range(0, num):
            self.fifo(states[i], actions[i], rewards[i], next_states[i])
    def sampling(self):
        indexes = np.random.randint(min(self.count, self.memory_size), size=self.batch_size)
        states = self.states[indexes]
        actions = self.actions[indexes]
        rewards = self.rewards[indexes]
        next_states = self.next_states[indexes]
        states = states.reshape((-1, self.state_dim))
        actions = actions.reshape((-1, 1))
        rewards = rewards.reshape((-1, 1))
        next_states = next_states.reshape((-1, self.state_dim))
        return states, actions, rewards, next_states

In [3]:
class Memory(object):
    def __init__(self, memory_size, state_dim, act_dim):
        self.memory_size = memory_size
        self.state_dim = state_dim
        self.act_dim = act_dim
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0
    def fifo(self, state, action, reward, next_state):
        self.states[self.current] = state
        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.next_states[self.current] = next_state
        self.current = (self.current + 1) % self.memory_size
        self.count = self.count + 1
    def add_trajectory(self, states, actions, rewards, next_states):
        num = len(observes)
        for i in range(0, num):
            self.fifo(states[i], actions[i], rewards[i], next_states[i])
    def sampling(self):
        randpermlist = np.random.permutation(self.count)
        states = self.states[randpermlist]
        actions = self.actions[randpermlist]
        rewards = self.rewards[randpermlist]
        next_states = self.next_states[randpermlist]
        states = states.reshape((-1, self.state_dim))
        actions = actions.reshape((-1, 1))
        rewards = rewards.reshape((-1, 1))
        next_states = next_states.reshape((-1, self.state_dim))
        return states, actions, rewards, next_states
    def empty_memory(self):
        self.states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.actions = np.empty((self.memory_size, 1), dtype=np.float32)
        self.rewards = np.empty(self.memory_size, dtype=np.float32)
        self.next_states = np.empty((self.memory_size, self.state_dim), dtype=np.float32)
        self.count = 0
        self.current = 0

In [4]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr=0.0005):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.a = tf.placeholder(tf.float32, (None, n_actions), "act")
        self.td_error = tf.placeholder(tf.float32, (None, 1), "td_error")  # TD_error]
        self.eps = 1
        self.eps_decay_rate = 0.9
        self.eps_min = 0

        with tf.variable_scope('Actor'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=64,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            l2 = tf.layers.dense(
                inputs=l1,
                units=32,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l2'
            )
            
            self.acts_prob = tf.layers.dense(
                inputs=l2,
                units=n_actions,    # output units
                activation=tf.contrib.sparsemax.sparsemax,   # get action probabilities
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='acts_prob'
            )
        '''    
        with tf.variable_scope('exp_v'):
            log_prob = tf.log(tf.reduce_sum(
                tf.multiply(self.acts_prob, self.a), 1, keep_dims = True))
            self.loss = tf.reduce_mean(log_prob * self.td_error)  # advantage (TD_error) guided loss

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(-self.loss)
        '''
        with tf.variable_scope('exp_v'):
            self.error = self.td_error + ALPHA*0.5*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.acts_prob), 1, keep_dims=True) + ALPHA*0.5 - ALPHA*tf.reduce_sum(
                tf.multiply(self.acts_prob, self.a), 1, keep_dims = True)
            # TD error + ALPHA(0.5*pi^2 + 0.5 - pi)
            self.loss = tf.reduce_sum(tf.square(self.error))

        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)  
            
    def learn(self, s, a, td):
        BATCH_SIZE = 200
        NITER = int(s.shape[0]/BATCH_SIZE)
        for i in range(0, NITER):
            feed_dict = {self.s: s[i*BATCH_SIZE:(i+1)*BATCH_SIZE], 
                         self.a: a[i*BATCH_SIZE:(i+1)*BATCH_SIZE], 
                         self.td_error: td[i*BATCH_SIZE:(i+1)*BATCH_SIZE]}
            _, loss = self.sess.run([self.train_op, self.loss], feed_dict)
        return 0

    def choose_action(self, s):
        s = s[np.newaxis, :]
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        probs = probs.ravel()
        probs = np.ones(probs.shape[0]) * self.eps / probs.shape[0] + probs * (1. - self.eps)
        probs = probs/np.sum(probs)
        return np.random.choice(np.arange(probs.shape[0]), p=probs)   # return a int
    
    def update_policy(self):
        self.eps = np.max((self.eps*self.eps_decay_rate, self.eps_min))
        
    def print_prob(self, s):
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        probs = probs.ravel()
        probs = np.ones(probs.shape[0]) * self.eps / probs.shape[0] + probs * (1. - self.eps)
        probs = probs/np.sum(probs)
        print(probs)
        return probs

In [ ]:
class Critic(object):
    def __init__(self, sess, n_features, lr=0.01):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, (None, n_features), "state")
        self.v_ = tf.placeholder(tf.float32, (None, 1), "v_next")
        self.r = tf.placeholder(tf.float32, (None, 1), 'r')

        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=32,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )
            
            l2 = tf.layers.dense(
                inputs=l1,
                units=16,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l2'
            )

            self.v = tf.layers.dense(
                inputs=l2,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                # bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )
            
        with tf.variable_scope('squared_TD_error'):
            self.td_error = self.r + GAMMA * self.v_ - self.v
            self.loss = tf.reduce_sum(tf.square(self.td_error))    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    def learn(self, s, r, s_):
        BATCH_SIZE = 200
        NITER = int(s.shape[0]/BATCH_SIZE)
        for i in range(0, NITER):
            v_ = self.sess.run(self.v, {self.s: s_[i*BATCH_SIZE:(i+1)*BATCH_SIZE]})
            td_error, _ = self.sess.run([self.td_error, self.train_op],
                                              {self.s: s[i*BATCH_SIZE:(i+1)*BATCH_SIZE], self.v_: v_, 
                                               self.r: r[i*BATCH_SIZE:(i+1)*BATCH_SIZE]})
        return 0
    
    def get_error(self, s, r, s_):
        v_ = self.sess.run(self.v, {self.s: s_})
        td_error = self.sess.run(self.td_error,
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error
        

In [ ]:
sess = tf.Session()

actor = Actor(sess, n_features=N_F, n_actions=N_A, lr=LR_A)
# critic = Critic(sess, n_features=N_F, lr=LR_C)
critic = Critic(sess, n_features=N_F, lr=LR_C)     # we need a good teacher, so the teacher should learn faster than the actor
MEMORY_SIZE = 10000
BATCH_SIZE = 1000
memory = Memory(MEMORY_SIZE, N_F, N_A)
# replay = ExpReplay(MEMORY_SIZE, N_F, N_A, BATCH_SIZE)
sess.run(tf.global_variables_initializer())

if OUTPUT_GRAPH:
    tf.summary.FileWriter("logs/", sess.graph)

for i_episode in range(MAX_EPISODE):
    s = env.reset()
    t = 0
    track_r = []
    while True:
        if i_episode % 5 == 0: RENDER = True
        if RENDER: env.render()

        a = actor.choose_action(s)
        s_, r, done, info = env.step([action_map[a]])

        track_r.append(r)
       
        #td_error = critic.learn(s, r, s_)  # gradient = grad[r + gamma * V(s_) - V(s)]
        #actor.learn(s, a, td_error)     # true_gradient = grad[logPi(s,a) * td_error]
        memory.fifo(s, a, r, s_)
        s = s_
        t += 1
        if done or t >= MAX_EP_STEPS: #done or
            RENDER = False
            ep_rs_sum = sum(track_r)
            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break
        
    if (i_episode + 1)%5 == 0:
        states, actions, rewards, next_states = memory.sampling()
        mx_actions = np.zeros((actions.shape[0], N_A))
        for i in range(0, states.shape[0]):
            mx_actions[i, int(actions[i, 0])] = 1
        td_errors = critic.get_error(states, rewards, next_states)
        actor.learn(states, mx_actions, td_errors)
        critic.learn(states, rewards, next_states)
        actor.update_policy()
        memory.empty_memory()

('episode:', 0, '  reward:', -1313)
('episode:', 1, '  reward:', -1321)
('episode:', 2, '  reward:', -1320)
('episode:', 3, '  reward:', -1334)
('episode:', 4, '  reward:', -1332)
('episode:', 5, '  reward:', -1338)
('episode:', 6, '  reward:', -1349)
('episode:', 7, '  reward:', -1360)
('episode:', 8, '  reward:', -1340)
('episode:', 9, '  reward:', -1336)
('episode:', 10, '  reward:', -1313)
('episode:', 11, '  reward:', -1308)
('episode:', 12, '  reward:', -1331)
('episode:', 13, '  reward:', -1336)
('episode:', 14, '  reward:', -1336)
('episode:', 15, '  reward:', -1354)
('episode:', 16, '  reward:', -1352)
('episode:', 17, '  reward:', -1341)
('episode:', 18, '  reward:', -1322)
('episode:', 19, '  reward:', -1334)
('episode:', 20, '  reward:', -1346)
('episode:', 21, '  reward:', -1345)
('episode:', 22, '  reward:', -1317)
('episode:', 23, '  reward:', -1333)
('episode:', 24, '  reward:', -1321)
('episode:', 25, '  reward:', -1324)
('episode:', 26, '  reward:', -1316)
('episode:'

('episode:', 220, '  reward:', -1286)
('episode:', 221, '  reward:', -1282)
('episode:', 222, '  reward:', -1277)
('episode:', 223, '  reward:', -1266)
('episode:', 224, '  reward:', -1280)
('episode:', 225, '  reward:', -1265)
('episode:', 226, '  reward:', -1266)
('episode:', 227, '  reward:', -1271)
('episode:', 228, '  reward:', -1270)
('episode:', 229, '  reward:', -1282)
('episode:', 230, '  reward:', -1272)
('episode:', 231, '  reward:', -1300)
('episode:', 232, '  reward:', -1319)
('episode:', 233, '  reward:', -1313)
('episode:', 234, '  reward:', -1323)
('episode:', 235, '  reward:', -1333)
('episode:', 236, '  reward:', -1346)
('episode:', 237, '  reward:', -1339)
('episode:', 238, '  reward:', -1362)
('episode:', 239, '  reward:', -1371)
('episode:', 240, '  reward:', -1368)
('episode:', 241, '  reward:', -1383)
('episode:', 242, '  reward:', -1386)
('episode:', 243, '  reward:', -1371)
('episode:', 244, '  reward:', -1371)
('episode:', 245, '  reward:', -1369)
('episode:',

('episode:', 436, '  reward:', -1703)
('episode:', 437, '  reward:', -1700)
('episode:', 438, '  reward:', -1699)
('episode:', 439, '  reward:', -1698)
('episode:', 440, '  reward:', -1706)
('episode:', 441, '  reward:', -1715)
('episode:', 442, '  reward:', -1716)
('episode:', 443, '  reward:', -1708)
('episode:', 444, '  reward:', -1698)
('episode:', 445, '  reward:', -1702)
('episode:', 446, '  reward:', -1692)
('episode:', 447, '  reward:', -1699)
('episode:', 448, '  reward:', -1691)
('episode:', 449, '  reward:', -1699)
('episode:', 450, '  reward:', -1694)
('episode:', 451, '  reward:', -1691)
('episode:', 452, '  reward:', -1688)
('episode:', 453, '  reward:', -1684)
('episode:', 454, '  reward:', -1686)
('episode:', 455, '  reward:', -1693)
('episode:', 456, '  reward:', -1683)
('episode:', 457, '  reward:', -1677)
('episode:', 458, '  reward:', -1675)
('episode:', 459, '  reward:', -1683)
('episode:', 460, '  reward:', -1684)
('episode:', 461, '  reward:', -1687)
('episode:',

('episode:', 653, '  reward:', -1714)
('episode:', 654, '  reward:', -1715)
('episode:', 655, '  reward:', -1716)
('episode:', 656, '  reward:', -1712)
('episode:', 657, '  reward:', -1711)
('episode:', 658, '  reward:', -1712)
('episode:', 659, '  reward:', -1716)
('episode:', 660, '  reward:', -1718)
('episode:', 661, '  reward:', -1708)
('episode:', 662, '  reward:', -1712)
('episode:', 663, '  reward:', -1708)
('episode:', 664, '  reward:', -1704)
('episode:', 665, '  reward:', -1707)
('episode:', 666, '  reward:', -1706)
('episode:', 667, '  reward:', -1709)
('episode:', 668, '  reward:', -1712)
('episode:', 669, '  reward:', -1713)
('episode:', 670, '  reward:', -1707)
('episode:', 671, '  reward:', -1710)
('episode:', 672, '  reward:', -1709)
('episode:', 673, '  reward:', -1713)
('episode:', 674, '  reward:', -1716)
('episode:', 675, '  reward:', -1712)
('episode:', 676, '  reward:', -1697)
('episode:', 677, '  reward:', -1700)
('episode:', 678, '  reward:', -1703)
('episode:',

('episode:', 869, '  reward:', -1648)
('episode:', 870, '  reward:', -1644)
('episode:', 871, '  reward:', -1634)
('episode:', 872, '  reward:', -1636)
('episode:', 873, '  reward:', -1634)
('episode:', 874, '  reward:', -1636)
('episode:', 875, '  reward:', -1633)
('episode:', 876, '  reward:', -1636)
('episode:', 877, '  reward:', -1632)
('episode:', 878, '  reward:', -1633)
('episode:', 879, '  reward:', -1633)
('episode:', 880, '  reward:', -1634)
('episode:', 881, '  reward:', -1635)
('episode:', 882, '  reward:', -1635)
('episode:', 883, '  reward:', -1636)
('episode:', 884, '  reward:', -1638)
('episode:', 885, '  reward:', -1638)
('episode:', 886, '  reward:', -1639)
('episode:', 887, '  reward:', -1640)
('episode:', 888, '  reward:', -1642)
('episode:', 889, '  reward:', -1644)
('episode:', 890, '  reward:', -1643)
('episode:', 891, '  reward:', -1634)
('episode:', 892, '  reward:', -1636)
('episode:', 893, '  reward:', -1631)
('episode:', 894, '  reward:', -1632)
('episode:',

In [ ]:
for i_episode in range(0, 200):
    s = env.reset()
    t = 0
    track_r = []
    while True:
        env.render()

        a = actor.choose_action(s)
        s_, r, done, info = env.step(action_map[a, :])
        s = s_
        t += 1
        if t >= MAX_EP_STEPS: #done or
            ep_rs_sum = sum(track_r)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break